In [ ]:
import numpy as np
import cv2
import configparser

### Feature detecting

In [ ]:
def feature_detect(src,minHessian=200,thresh=0,is_setUpright=True):
    row_indices,col_indices = np.nonzero(src)
    points = (col_indices,row_indices)
    points_np = np.array(points).T

    e1= cv2.fitEllipse(points_np)
    center_x = int(e1[0][0])
    src_left = src[:,:center_x]
    src_right = src[:,center_x:]
    if(np.count_nonzero(src_left) == 0 or np.count_nonzero(src_right) == 0):
        print("entropy p")
        return src,(col_indices,row_indices)

    # Step 1: Detect the keypoints using SURF Detector
    detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    detector.setUpright(is_setUpright)
    
    keypoints_left = detector.detect(src_left)
    pts_left = np.array([np.around(keypoints_left[idx].pt) for idx in range(len(keypoints_left))], dtype = int)
    
    keypoints_right = detector.detect(src_right)
    pts_right = np.array([np.around(keypoints_right[idx].pt) for idx in range(len(keypoints_right))], dtype = int)

    src_left_new = np.zeros((src_left.shape[0], src_left.shape[1]), dtype=np.uint8)
    src_right_new = np.zeros((src_right.shape[0], src_right.shape[1]), dtype=np.uint8)

    if(len(pts_left)> thresh):
        pts_left_x_min,pts_left_x_max = np.min(pts_left[:,0]),np.max(pts_left[:,0])
        pts_left_y_min,pts_left_y_max = np.min(pts_left[:,1]),np.max(pts_left[:,1])
        
        src_left_new[pts_left_y_min:pts_left_y_max,pts_left_x_min:pts_left_x_max] = src_left[pts_left_y_min:pts_left_y_max,pts_left_x_min:pts_left_x_max]
        if(np.count_nonzero(src_left_new) == 0):
            src_left_new = src_left
    else:
        src_left_new = src_left
    
    if(len(pts_right)> thresh):
        # Step 2: Find the xmin xmax ymin ymax for pts_left, pts_righ
        pts_right_x_min,pts_right_x_max = np.min(pts_right[:,0]),np.max(pts_right[:,0])
        pts_right_y_min,pts_right_y_max = np.min(pts_right[:,1]),np.max(pts_right[:,1])

        src_right_new[pts_right_y_min:pts_right_y_max,pts_right_x_min:pts_right_x_max] = src_right[pts_right_y_min:pts_right_y_max,pts_right_x_min:pts_right_x_max]
        if(np.count_nonzero(src_right_new) == 0):
            src_right_new = src_right
    else:
        src_right_new = src_right
        
    
    src_new = np.zeros((src.shape[0], src.shape[1]), dtype=np.uint8)
    src_new = np.hstack((src_left_new,src_right_new))
    row_indices,col_indices = np.nonzero(src_new)

    return src_new,(col_indices,row_indices)

### Feature matching

In [ ]:
def find_center_x(img):
    row_indices,col_indices = np.nonzero(img)
    points = (col_indices,row_indices)
    points_np = np.array(points).T
    e1= cv2.fitEllipse(points_np)
    center_x = int(e1[0][0])
    return center_x

def is_feature_matching(img1_l_r,img2_l_r,minHessian=120,thresh=5): 
    flag = False
    # Step 1: Detect the keypoints using SURF Detector, compute the descriptors
    detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    detector.setUpright(True)
    kp1, des1 = detector.detectAndCompute(img1_l_r, None)
    kp2, des2 = detector.detectAndCompute(img2_l_r, None)
    if(len(kp1)==0 or len(kp2)==0):
        return flag,kp1,kp2
    
    # Step 2: Matching descriptor vectors with a brute force matcher
    matcher = cv2.BFMatcher.create(cv2.NORM_L1, crossCheck=True)
    matches = matcher.match(des1, des2)
    if(len(matches) >= thresh):
        flag = True
    
    # Step 3: Calculate new keypoint.pt for img1_l_r and img2_l_r
    pt1 = []
    pt2 = []
    for i,m in enumerate(matches):
        pt1.append(np.around(kp1[m.queryIdx].pt))
        pt2.append(np.around(kp2[m.trainIdx].pt))
    
    pt1 = np.array(pt1,dtype=int)
    pt2 = np.array(pt2,dtype=int)
    
    return flag,pt1,pt2

def is_feature_matching_all(img1,img2): # Check the matching of the left and right parts separately
    center_x = find_center_x(img1)
    img1_left = img1[:,:center_x]
    img1_right = img1[:,center_x:]
    if(np.count_nonzero(img1_left) == 0 or np.count_nonzero(img1_right) == 0):
        return False,0,0,False,0,0
    
    center_x = find_center_x(img2)
    img2_left = img2[:,:center_x]
    img2_right = img2[:,center_x:]
    if(np.count_nonzero(img2_left) == 0 or np.count_nonzero(img2_right) == 0):
        return False,0,0,False,0,0
    
    flag_l,pt1_l,pt2_l = is_feature_matching(img1_left,img2_left)
    flag_r,pt1_r,pt2_r = is_feature_matching(img1_right,img2_right)
    return flag_l,pt1_l,pt2_l,flag_r,pt1_r,pt2_r
    
def cal_bb(pt1,pt2):
    pt1_x_min,pt1_x_max = np.min(pt1[:,0]),np.max(pt1[:,0])
    pt1_y_min,pt1_y_max = np.min(pt1[:,1]),np.max(pt1[:,1])

    pt2_x_min,pt2_x_max = np.min(pt2[:,0]),np.max(pt2[:,0])
    pt2_y_min,pt2_y_max = np.min(pt2[:,1]),np.max(pt2[:,1])
    
    return (pt1_x_min,pt1_x_max,pt1_y_min,pt1_y_max),(pt2_x_min,pt2_x_max,pt2_y_min,pt2_y_max)    

def feature_remove(img1,img2,pt1_l,pt2_l,pt1_r,pt2_r):
    center_x = find_center_x(img1)
    img1_left = img1[:,:center_x]
    img1_right = img1[:,center_x:]
    
    center_x = find_center_x(img2)
    img2_left = img2[:,:center_x]
    img2_right = img2[:,center_x:]
    
    # Step 4: Calculate the bounding box of keypoint.pt for img1 and img2 
    img1_left_bb,img2_left_bb = cal_bb(pt1_l,pt2_l)
    img1_right_bb,img2_right_bb = cal_bb(pt1_r,pt2_r)
    
    img1_left_new = np.zeros((img1_left.shape[0], img1_left.shape[1]), dtype=np.uint8)
    img1_left_new[img1_left_bb[2]:img1_left_bb[3],img1_left_bb[0]:img1_left_bb[1]] = img1_left[img1_left_bb[2]:img1_left_bb[3],img1_left_bb[0]:img1_left_bb[1]]
    img1_right_new = np.zeros((img1_right.shape[0], img1_right.shape[1]), dtype=np.uint8)
    img1_right_new[img1_right_bb[2]:img1_right_bb[3],img1_right_bb[0]:img1_right_bb[1]] = img1_right[img1_right_bb[2]:img1_right_bb[3],img1_right_bb[0]:img1_right_bb[1]]
    
    img1_new = np.zeros((img1.shape[0], img1.shape[1]), dtype=np.uint8)
    img1_new = np.hstack((img1_left_new,img1_right_new))
    
    img2_left_new = np.zeros((img2_left.shape[0], img2_left.shape[1]), dtype=np.uint8)
    img2_left_new[img2_left_bb[2]:img2_left_bb[3],img2_left_bb[0]:img2_left_bb[1]] = img2_left[img2_left_bb[2]:img2_left_bb[3],img2_left_bb[0]:img2_left_bb[1]]
    img2_right_new = np.zeros((img2_right.shape[0], img2_right.shape[1]), dtype=np.uint8)
    img2_right_new[img2_right_bb[2]:img2_right_bb[3],img2_right_bb[0]:img2_right_bb[1]] = img2_right[img2_right_bb[2]:img2_right_bb[3],img2_right_bb[0]:img2_right_bb[1]]
    
    img2_new = np.zeros((img2.shape[0], img2.shape[1]), dtype=np.uint8)
    img2_new = np.hstack((img2_left_new,img2_right_new))
    
    row_indices1,col_indices1 = np.nonzero(img1_new)
    row_indices2,col_indices2 = np.nonzero(img2_new)
    
    return img1_new,(col_indices1,row_indices1),img2_new,(col_indices2,row_indices2)

### Ellipse Fitting

In [ ]:
def rect_transform(box, trans):
    ellipse = (((box[0][0]+trans[1])/trans[0],(box[0][1]+trans[2])/trans[0]),
        (box[1][1]/trans[0],box[1][0]/trans[0]),box[2]+90)
    return ellipse

In [ ]:
def write_ellipse_params(e,f):
    write_config = configparser.ConfigParser()
    write_config.add_section("iris")
    write_config.set("iris","exist","true")
    write_config.set("iris","center_x",str(e[0][0]))
    write_config.set("iris","center_y",str(e[0][1]))
    write_config.set("iris","short_radius",str(e[1][1]/2))
    write_config.set("iris","long_radius",str(e[1][0]/2))
    write_config.set("iris","rad_phi",str(e[2]))

    cfgfile = open(f,'w')
    write_config.write(cfgfile)
    cfgfile.close()